In [1]:
# Generic libraries
# Reference material - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
import pandas as pd
import numpy  as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Flatten
from sklearn.metrics import mean_squared_error

# Plotting
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Making the plots standard 
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 5]

In [2]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [3]:
# read the data 
filtered_df = pd.read_pickle('ec2_allprobes.pickle')
normalised_df = pd.read_pickle('normalised_df.pickle')
normalised_df

normalized_datetime  normal_prbid  normalized_avg   
dst_addr                                                                
52.46.200.93 22540             0.000106             5       -0.535096  \
             22541             0.000000            10        2.299934   
             22542             0.000106            11        1.660160   
             22543             0.000438            12        0.413936   
             22544             0.000361             8       -0.892898   
...                                 ...           ...             ...   
             28170             0.999740             4       -0.281200   
             28171             0.999692             9       -0.246793   
             28172             0.999515             2       -0.472978   
             28173             0.999799             6       -1.040529   
             28174             1.000000             1       -0.584520   

                    normalized_distance  classes  
dst_addr                                          
52.46.200.93 22540             0.202324        0  
             22541             1.000000        2  
             22542             0.745771        2  
             22543             0.553591        1  
             22544             0.011941        0  
...                                 ...      ...  
             28170             0.267027        0  
             28171             0.554507        0  
             28172             0.228537        0  
             28173             0.000000        0  
             28174             0.231181        0  

[5587 rows x 5 columns]

In [4]:
features = normalised_df["normalized_distance"]
target = normalised_df["normalized_avg"]

# Calculate the index to split the data
split_index = int(len(normalised_df) * 0.75)


# split the dataset into training and testing sets
X_train = features[0:int(split_index)].values
X_test =  features[split_index :]
y_train = target[0:int(split_index)]
y_test = target[split_index :]

In [5]:
# split into samples (e.g. 4190/10 )
# split a univariate sequence into samples
from numpy import array

def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return array(X)

Xtrain_3d = split_sequence(X_train,10)
Xtest_3d = split_sequence(X_test,10)
Xtrain_3d.shape

(4180, 10)

In [6]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
Xtrain_3d  = Xtrain_3d .reshape((Xtrain_3d.shape[0], Xtrain_3d.shape[1], n_features))
Xtest_3d = Xtest_3d .reshape((Xtest_3d.shape[0], Xtest_3d.shape[1], n_features))
Xtrain_3d[0].shape

(10, 1)

In [7]:
# define model
n_steps = 10 

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
# fit model
model.fit(Xtrain_3d, y_train[0:len(Xtrain_3d)], epochs=200, verbose=0)

2023-06-30 16:23:30.074579: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [9]:
y_pred = model.predict(Xtest_3d, verbose=0)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_pred,y_test[0:len(y_pred)])
mse

0.01067332102692572